In [1]:
'''
Script to preprocess the data 
'''
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels
from scipy.stats import mannwhitneyu, wilcoxon
import itertools
from sklearn.preprocessing import StandardScaler
from numpy import inf
import math
from statsmodels.stats.multitest import multipletests
from matplotlib.pyplot import cm
from sklearn.cluster import AgglomerativeClustering


df = pd.read_csv('/Users/tlh4362/Desktop/final_dataset_10_07.csv')
# seq = pd.read_csv('/Users/tlh4362/Downloads/seq_final_list(updated).csv')
# labs = pd.read_csv('/Users/tlh4362/Desktop/labs_9_1.csv')
labs = pd.read_csv('/Users/tlh4362/Desktop/labs_10_10.csv')
ct = pd.read_csv('/Users/tlh4362/Desktop/SARSCoV2Biobank-QPCRct_DATA_2022-08-29_2050.csv')

risk = pd.read_csv('/Users/tlh4362/Desktop/risk_scores.csv')
raw_risk = pd.read_csv('/Users/tlh4362/Downloads/SARSCoV2Biobank-Justriskscores_DATA_2022-10-21_1314.csv')
test1 = pd.read_csv('/Users/tlh4362/Desktop/IDPHExport20220405043254528.csv')
test2 = pd.read_csv('/Users/tlh4362/Desktop/IDPHExport20220405043256767.csv')

labs = labs.drop('clade', axis = 1)

df['sample_date'] = pd.to_datetime(df['sample_date'])


merged = pd.merge(test1, test2, on = 'ReportDate')

merged['cases'] = merged.filter(regex = 'CasesChange').sum(axis = 1)
merged['tests'] = merged.filter(regex = 'TestedChange').sum(axis = 1)
merged['sample_date'] = merged['ReportDate']
merged['positivity_rate'] =positivity_rate = merged.filter(regex = 'CasesChange').sum(axis = 1)/merged.filter(regex = 'TestedChange').sum(axis = 1)
merged.at[merged[merged['positivity_rate'] == inf].index, 'positivity_rate'] = None


df['sample_date'] = pd.to_datetime(df['sample_date'])
merged['sample_date'] = pd.to_datetime(merged['sample_date'])
labs['sample_date'] = pd.to_datetime(labs['sample_date'])

merged['normalized_cases'] =  merged['cases']/merged['cases'].max()

daily_seq_counts = {}
normalized_counts = {}
tmp = df[df['clade'].notnull()]
for date in tmp['sample_date'].unique():
    daily_seq_counts[date] = tmp[tmp['sample_date']==date].shape[0]

for d, c in daily_seq_counts.items():
    normalized_counts[d] = c/max(daily_seq_counts.values())

normalized_counts = pd.DataFrame().from_dict(normalized_counts, orient  = 'index', columns = ['normalized_sequences'])
normalized_counts['sample_date'] = pd.to_datetime(normalized_counts.index)

df = pd.merge(df, normalized_counts, on = 'sample_date', how = 'left')

df = pd.merge(df, merged.loc[:,['cases', 'tests','sample_date', 'positivity_rate', 'normalized_cases']], on = 'sample_date', how = 'left')

df['seq_case_ratio'] = df['normalized_sequences']/df['normalized_cases']




# df = df.drop(df[(df['ca_hosp_admit']==0) &(df['ca_death']==1)].index)

#converting race labels 
df['ca_race'] = df['ca_race'].replace({
1:'White',
2:'Black or African American',
3:'Race_Other', #'Native Hawaiian or Other Pacific Islander'
4:'Asian',
5:'Race_Other', #'American Indian or Alaska Native'
6:'Race_Declined_Unknown',#'Race_Unknown'
7:'Race_Other',
8:'Race_Declined_Unknown', #Race_Unable to Answer
9:'Race_Declined_Unknown'}) #Race_Declined

#converting ethnicity labels 
df['ca_ethnicity'] = df['ca_ethnicity'].replace({
    1:'Not Hispanic or Latino', 
    2: 'Hispanic or Latino', 
    3:'Ethnicity_Declined', 
    4:'Ethnicity_Unknown'})

df['sex'] = df['sex'].replace({1:'Female',2:'Male',3:'Unknown'})
df['sex'] = df['sex'].fillna('Unknown')
df['ca_race'] = df['ca_race'].fillna('Race_Other')

# df = pd.merge(df, risk, on = 'record_id', how = 'left')
df = pd.merge(df, labs, on = ['mrn', 'sample_date'], how = 'left')
df['ca_lab_asltr'] = df['ca_lab_ast']/df['ca_lab_alt']

# df.at[df[df['ca_lab_asltr'] == inf].index, 'ca_lab_asltr'] = None

#clinical outcome calculated as if there was hospital admission, ICU admission or death.
df['clinical_outcome'] = [1 if df.loc[idx, ['ca_hosp_admit', 'ca_icu', 'ca_death']].sum() > 0 else 0 for idx in df.index ]


#BMI had commas, need to convert to correct format. 
new_bmi = []
for x in df['ca_bmi']:
    if isinstance(x, str):
        if len(x.split(','))==1:
             new_bmi.append(float(x.split(',')[0]))
        else:
            new_bmi.append(float('{}.{}'.format(x.split(',')[0], x.split(',')[1])))
    else:
        new_bmi.append(None)

df['ca_bmi'] = new_bmi

df['clade']= df['clade'].replace({
    '21J (Delta)' : '21J/I/A (Delta)',
    '21I (Delta)' : '21J/I/A (Delta)',
    '21A (Delta)' : '21J/I/A (Delta)',
    '21K (Omicron)' : '21M/K/L (Omicron)',
    '21L (Omicron)' : '21M/K/L (Omicron)',
    '21M (Omicron)' : '21M/K/L (Omicron)',
    '20J (Gamma, V3)': 'Other',
    '21D (Eta)'     : 'Other',
    'recombinant'   : 'Other',
    '21C (Epsilon)' : 'Other',     
    '19B'           : 'Other',      
    '21F (Iota)'    : 'Other',
    '21H (Mu)'      : 'Other',       
    '20H (Beta, V2)': 'Other',       
    '19A'           : 'Other',     
    '21G (Lambda)'  : 'Other',      
    '20D'           : 'Other',
    '19B'           : 'Other'
})

df = df.rename({'ca_icu':'ICU',
'age':'Age',
'clade':'Clade',
'sex':'Sex',
'ca_race':'Race',
'ca_ethnicity':'Ethnicity',
'ca_comorbid_sum':'Number_of_Comorbidities',
'ca_bmi':'BMI',
'doses_before_infection':'Number_of_vaccination_doses_before_infection'}, axis = 1)

# dropping demographics that don't reach 1% of the total sample size
demo = ['Race', 'Sex'] #'Sex'
to_drop_idx = []
for d in demo:
    proportion = df[d].value_counts()/df.shape[0]
    for drop in proportion[proportion<0.05].index:
        print(drop, d, df[df[d]==drop]['Clade'].value_counts())
        to_drop_idx.extend(list(df[df[d]==drop].index))
print(df.shape)

df = df.drop(to_drop_idx)

print(df.shape)


/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (8,18,28,51,53) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(14252, 255)
(14252, 255)


In [3]:
df.to_csv('/Users/tlh4362/Desktop/final_dataset_analysis_11_2.csv')